# LLM DETECTION

In [8]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier

In [3]:
train = pd.read_csv('open/train.csv', encoding='utf-8')
test = pd.read_csv('open/test.csv', encoding='utf-8')

In [14]:
print(train[train['generated'] == 1].count())

title        7995
full_text    7995
generated    7995
dtype: int64


In [6]:
X = train[['title', 'full_text']]
y = train['generated']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
get_title = FunctionTransformer(lambda x: x['title'], validate=False)
get_text = FunctionTransformer(lambda x: x['full_text'], validate=False)

vectorizer = FeatureUnion([
    ('title', Pipeline([('selector', get_title), 
                        ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=3000))])),
    ('full_text', Pipeline([('selector', get_text),
                        ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=10000))])),
])

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

In [ ]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train_vec, y_train)

val_probs = xgb.predict_proba(X_val_vec)[:, 1]
auc = roc_auc_score(y_val, val_probs)
print(f'Validation AUC: {auc:.4f}')

Validation AUV: 0.9183


In [18]:
X_val_copy = X_val.copy()
X_val_copy['predicted'] = val_probs
X_val_copy

,title,full_text,predicted
71026,Santana IV,"이 음반은 카를로스 산타나, 그렉 롤리, 닐 숀, 마이클 카라벨로, 마이클 슈리브를...",0.123861
16801,다니엘 루이스바산,"다니엘 루이스바산 후스타(, 1951년 6월 28일, 바스크 주 소푸에르타 ~ )는...",0.104320
51979,UZ HRCS2,HRCS2는 유로 2012를 앞두고 도입된 현대로템에서 생산한 우크라이나의 전동차이...,0.007306
11014,비키니 왁싱,비키니 왁싱(bikini waxing) 또는 브라질리언 왁싱(brazilian wa...,0.004940
7932,A-A선,"아르한겔스크-아스트라한 선, 또는 A-A 선은 바르바로사 작전 당시 군사 목표 지점...",0.038887
...,...,...,...
62049,탁스코,탁스코는 멕시코 게레로주에 위치한 작은 도시로 면적은 347km2 이고 해발고도 1...,0.018463
87373,이고리 쿠르차토프,"이고리 바실리예비치 쿠르차토프(, 1903년 1월 12일 ~ 1960년 2월 7일)...",0.050209
9902,국립사이버안보센터,"1951년, 영국 국방부가 위치한 런던의 이스트코트에 GCHQ 본부가 있었는데, 첼...",0.046668
79629,클라리스 스탈링,클라리스 M. 스탈링(Clarice M. Starling)은 미국의 소설가 토머스 ...,0.996990


In [22]:
# test용으로 'paragraph_text'를 'full_text'에 맞게 재명명
test = test.rename(columns={'paragraph_text': 'full_text'})

X_test = test[['title', 'full_text']]

X_test_vec = vectorizer.transform(X_test)

probs = xgb.predict_proba(X_test_vec)[:, 1]

In [27]:
sample_submission = pd.read_csv('./open/sample_submission.csv', encoding='utf-8-sig')

sample_submission['generated'] = (probs >= 0.05).astype(int)

sample_submission.to_csv(f'./baseline_submission.csv', index=False)